In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

AGRUPACIONES_ESPECIALIDAD = {
    "Broncopulmonar": [
        "AVNIA",
        "BRONCOPULMONAR ADULTO INT",
        "BRONQUIECTACIAS",
        "COMPIN",
        "DERRAME PLEURAL",
        "EPOC TIOTROPIO",
        "ENFERMEDADES PROFESIONALES",
        "EX. BRONCOSCOPIA",
        "FIBROSIS PULMONAR",
        "GES ASMA",
        "GES EPOC",
        "GES FIBROSIS QUISTICA",
        "HIPERTENSION PULMONAR",
        "INGRESO PROGRAMA DE OXIGENO",
        "OXIGENOTERAPIA",
        "PESQUISA CANCER PULMONAR",
        "PRIORITARIO BRONCOPULMONAR",
        "PULMON REUMATOLÓGICO",
        "TABACO GRUPAL",
        "TABACO INDIVIDUAL",
        "TBC",
    ],
    "Cardiocirugia": [
        "CARDIOCIRUGIA",
        "CARDIOPATIA CONGENITA",
        "TRASPLANTE CARDIACO",
        "PATOLOGÍA DE LA AORTA TORÁCICA-MARFAN",
    ],
    "Cirugia de Torax": [
        "CIRUGIA DE TORAX",
        "ONCOLOGIA",
        "TRASPLANTE PULMONAR",
    ],
    "Unidad del Sueno": [
        "UNIDAD DE SUEÑO",
        "UNIDAD DE SUEÑO-OTORRINOLARINGOLOGO",
    ],
    "Cardiologia": [
        "ARRITMIA",
        "CARDIOLOGIA",
        "EX. ECOCARDIO URGENCIA",
        "EX. ECOCARDIOGRAMA",
        "EX. HOLTER CONGENITOS INT",
        "EX. TEST DE ESFUERZO CONGENITO",
        "GES MARCAPASO",
        "GES MARCAPASO PRE QUIRÚRGICO",
        "DESFIBRILADORES / RESINCRONIZADORES",
        "ELECTROFISIOLOGIA",
    ],
    "Cuidados Paliativos": ["GES CUIDADOS PALIATIVOS"],
}

In [ ]:
# Lee base de datos Ambulatorios
df = pd.read_csv("../data/processed/his_procesada.csv")
df["nombre_especialidad"] = df["nombre_especialidad"].astype(str)

In [ ]:
# Renombra especialidades
df_especialidades_reemplazadas = df.copy()
for nuevo_valor, a_cambiar in AGRUPACIONES_ESPECIALIDAD.items():
    df_especialidades_reemplazadas["nombre_especialidad"] = df_especialidades_reemplazadas[
        "nombre_especialidad"
    ].replace(a_cambiar, nuevo_valor)

# Elimina consultas de enfermeria y examenes
df_especialidades_reemplazadas = df_especialidades_reemplazadas[
    ~df_especialidades_reemplazadas["nombre_especialidad"].str.contains(
        r"ENFERMERA|EX.", regex=True
    )
]

In [ ]:
# Obtiene el desglose del numero de consultas por diagnostico y tipo de especialidad
conteo_consultas_por_especialidad = (
    df_especialidades_reemplazadas.groupby(["ano", "codigo_diagnostico"])["nombre_especialidad"]
    .value_counts()
    .reset_index(name="n_consultas")
    .set_index(["ano", "codigo_diagnostico"])
)

# Obtiene los pacientes distintos por diagnostico
pacientes_por_diagnostico = df_especialidades_reemplazadas.groupby(["ano", "codigo_diagnostico"])[
    "id_paciente"
].nunique()

pacientes_por_tipo_de_consulta = conteo_consultas_por_especialidad.join(pacientes_por_diagnostico)
pacientes_por_tipo_de_consulta["fraccion_consulta"] = (
    pacientes_por_tipo_de_consulta["n_consultas"] / pacientes_por_tipo_de_consulta["id_paciente"]
)
pacientes_por_tipo_de_consulta = pacientes_por_tipo_de_consulta.reset_index()

In [ ]:
# Obtiene la cantidad de consultas y pacientes agrupados en el periodo
consultas_por_especialidad_agrupadas = (
    df_especialidades_reemplazadas.groupby(["codigo_diagnostico", "nombre_especialidad"])
    .agg(n_consultas=("nombre_especialidad", "count"), n_pacientes=("id_paciente", "nunique"))
    .reset_index()
)

# Calcula la proporcion de consultas por paciente
consultas_por_especialidad_agrupadas["consultas_por_paciente"] = (
    consultas_por_especialidad_agrupadas["n_consultas"]
    / consultas_por_especialidad_agrupadas["n_pacientes"]
)

# Redondea la cantidad de consultas por paciente
consultas_por_especialidad_agrupadas["consultas_por_paciente_redond"] = round(
    consultas_por_especialidad_agrupadas["consultas_por_paciente"]
)